# Geoinformatik-Programmierpraktikum
## Random Forest

# Praktische Aufgabe:
Die gestackte Landsat-Szene (ohne Indizes) soll mittels einer Random Forest Klassifikation in thematische Klassen eingeordnet werden, hierfür sind folgende Arbeitsschritte notwendig:
- Erzeuge in QGIS Trainingsdaten für die LULC Klassen (identisch zu den Klassen des Entscheidungsbaums vergangener Woche)
- Bereite die Trainingsdaten für die Klassifikation vor und Teile diese in Training und Test (Test in der kommenden Woche)
- Führe eine Klassifikation mittes Random Forest und eigener Auswahl der Parameter `n_estimators` und `max_features` durch
- Speicher das Ergebnis als Raster ab

In [1]:
raster_ganze_Szene = r"C:\Users\marvi\Geoinformatik_Programmierpraktikum\Daten\Baender_gestacked\all_bands.tif"
rasterized_output = r"C:\Users\marvi\Geoinformatik_Programmierpraktikum\Daten\Sitzung_3_rasterized_shp.tif"
punkte_shp = r"C:\Users\marvi\Geoinformatik_Programmierpraktikum\Daten\shapes\Sitzung_3_SVM\punkte_Sitzung_3.shp"
ergebnis_tif = r"C:\Users\marvi\Geoinformatik_Programmierpraktikum\Daten\Baender_gestacked\klassifiziert_ergebnis_Sitzung_3.tif"

In [2]:
from osgeo import gdal, ogr
def rasterize(rasterReference, shapefile, trainRaster, attribute):
 img = gdal.Open(rasterReference)
 shp= ogr.Open(shapefile)
 outDS = gdal.GetDriverByName('GTiff')
 outDS = outDS.Create(trainRaster, img.RasterXSize, img.RasterYSize,1, gdal.GDT_Byte)
 outDS.SetGeoTransform(img.GetGeoTransform())
 outDS.SetProjection(img.GetProjectionRef())
 gdal.RasterizeLayer(outDS,[1], shp.GetLayer(), options=[str("ATTRIBUTE=" + attribute)])
 outDS = None
#rasterize(r"originalStack.tif", r"trainData.shp",r"trainData.tif", "FeatureNameShp")

In [3]:
rasterize(raster_ganze_Szene, punkte_shp, rasterized_output, "class")

In [4]:
# training data
stack = gdal.Open(raster_ganze_Szene).ReadAsArray()
trainNtest = gdal.Open(rasterized_output).ReadAsArray()
# training spectra
train = stack[:, trainNtest > 0].transpose()
print(train.shape)
# training labels
labels = trainNtest[trainNtest > 0]
print(labels.shape)

(280, 7)
(280,)


In [5]:
from sklearn.model_selection import train_test_split
# train & test split
X_train, X_test, y_train, y_test = train_test_split(train, labels, train_size = 0.5, test_size= 0.5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(140, 7)
(140, 7)
(140,)
(140,)


In [6]:
 # Random Forest Model - Default Parameter
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(n_estimators = 100, max_features = "sqrt")

In [7]:
randomForest = randomForest.fit(X_train, y_train)

In [8]:
print(stack.shape)
# (7, 353, 529)
targetShape = (stack.shape[0], stack.shape[1] * stack.shape[2])
print(targetShape)
# (7, 186737)
stackShaped = stack.reshape(targetShape).transpose()
print(stackShaped.shape)

(7, 353, 529)
(7, 186737)
(186737, 7)


In [9]:
predict = randomForest.predict(stackShaped)
print(predict)

[7 7 3 ... 7 7 3]


In [10]:
classification = predict.reshape([stack.shape[1], stack.shape[2]])
print(classification.shape)

(353, 529)


In [13]:
from osgeo import osr
def array2raster(originalImage, newImage, array):

	raster = gdal.Open(originalImage)
	geotransform = raster.GetGeoTransform()
	originX = geotransform[0]
	originY = geotransform[3]
	pixelWidth = geotransform[1]
	pixelHeight = geotransform[5]
	cols = raster.RasterXSize
	rows = raster.RasterYSize

	driver = gdal.GetDriverByName('Gtiff')
	outRaster = driver.Create(newImage, cols, rows, 1, gdal.GDT_Float32)
	outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
	outband = outRaster.GetRasterBand(1)
	outband.WriteArray(array)
	outRasterSRS = osr.SpatialReference()
	outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
	outRaster.SetProjection(outRasterSRS.ExportToWkt())
	outband.FlushCache()

In [14]:
array2raster(raster_ganze_Szene, ergebnis_tif, classification)